# Segmenting and Clustering Neighborhoods in Toronto

importing libraries required

In [4]:
from bs4 import BeautifulSoup

import pandas as pd #library for data analsysis 
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /Users/dzl/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.1.0               |           py37_0         722 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         821 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/osx-64::altair-3.1.0-py37_0
  branca             conda-forge/noarch::branca-0.3.1-py_0
  folium             conda-forge/no

In [126]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

source = requests.get(url).text

soup = BeautifulSoup(source, 'lxml') #parse HTML

data = []
table = soup.find('table')
table_body = table.find('tbody')
colheaders = table.find_all('th')
colheaders = [ele.text.strip() for ele in colheaders]

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

# convert to pandas DataFrame
df = pd.DataFrame (data,columns=colheaders)
df.drop([0],axis=0,inplace=True)

# Ignore cells with a borough that is Not assigned.
df_noboro = df[df.Borough != 'Not assigned']

df_noboro.reset_index(drop=True,inplace=True)
df_noboro.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [127]:
#replace 'Not assigned' Neighbourhood with Borough Name

for i in enumerate(df_noboro['Neighbourhood']):
    if (i[1] == 'Not assigned'):
        df_noboro.iat[i[0],2]=df_noboro.iat[i[0],1]
        
df_noboro.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [156]:
# Combining Neighbourhoods with same Postcode into the same line, seperated by comma
df_to = df_noboro.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
df_to = df_to.reset_index()

In [157]:
df_to.shape

(103, 3)